# Logistic Regression and Gradient descent

## Loading data

Loading the Iris dataset from scikit-learn. 

In [ ]:
from sklearn import datasets
import numpy as np

In [ ]:
iris = datasets.load_iris()

Features:
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

Target:
- Iris Setosa
- Iris Versicolour
- Iris Virginica

In [ ]:
iris.data[0:5]

In [ ]:
iris.target

The classes are already converted to integer labels where 0=Iris-Setosa, 1=Iris-Versicolor, 2=Iris-Virginica.

This demo uses the third and four feature which are petal length and petal width.  Let's focus only on the first 2 classes for this demo.

In [ ]:
X = iris.data[:, [1,2]]
y = iris.target

print('Class labels:', np.unique(y))

X = X[y != 2,:]
y = y[y != 2]

print('Focus class labels:', np.unique(y))

Splitting data into 80% training and 20% test data:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

Standardizing the features:

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Training a logistic regression model with scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(X_train_std, y_train)

In [ ]:
lr.predict(X_test_std)

In [ ]:
lr.predict_proba(X_test_std)

predict() and predict_proba() method take a 2d array of values.

In [ ]:
lr.predict(X_test_std[0, :].reshape(1, -1))

In [ ]:
lr.predict_proba(X_test_std[0, :].reshape(1, -1))

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(y_test, lr.predict(X_test_std)))

### Decision boundary visualization

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import warnings


def versiontuple(v):
    return tuple(map(int, (v.split("."))))


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.6, 
                    c=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)

    # highlight test samples
    if test_idx:
        # plot all samples
        if not versiontuple(np.__version__) >= versiontuple('1.9.0'):
            X_test, y_test = X[list(test_idx), :], y[list(test_idx)]
            warnings.warn('Please update to NumPy 1.9.0 or newer')
        else:
            X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c=None,
                    alpha=1.0,
                    edgecolor='black',
                    linewidths=1,
                    marker='o',
                    s=55, label='test set')

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

plot_decision_regions(X_combined_std, y_combined,
                      classifier=lr, test_idx=range(len(y_train), len(y_combined)))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/logistic_regression.png', dpi=300)
plt.show()

### Regularization

C: Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

In [ ]:
weights, params, acc = [], [], []
for c in np.arange(-5., 5.):
    lr = LogisticRegression(C=10.**c, random_state=0)
    lr.fit(X_train_std, y_train)
    print(lr.coef_)
    weights.append(lr.coef_[0])
    params.append(10**c)
    acc.append(accuracy_score(y_test, lr.predict(X_test_std)))

In [ ]:
weights = np.array(weights)
plt.plot(params, weights[:, 0],
         label='petal length')
plt.plot(params, weights[:, 1], linestyle='--',
         label='petal width')
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.legend(loc='upper left')
plt.xscale('log')
# plt.savefig('./figures/regression_path.png', dpi=300)
plt.show()

In [ ]:
plt.plot(params, acc, label='test accuracy')
plt.ylabel('accuracy')
plt.xlabel('regularization constant')
plt.legend(loc='upper left')
plt.xscale('log')
# plt.savefig('./figures/regression_path.png', dpi=300)
plt.show()

Prediction has been affected

In [ ]:
lr.predict_proba(X_test_std)

In [ ]:
lr_l = LogisticRegression(C=10.**-5, random_state=0)
lr_l.fit(X_train_std, y_train)
lr_l.predict_proba(X_test_std)

<br>

## Gradient Decent for Logistic Regression

alpha : Constant that multiplies the regularization term

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", alpha = 0.001)
clf.fit(X_train_std, y_train)

print('Accuracy: %.2f' % accuracy_score(y_test, clf.predict(X_test_std)))

plot_decision_regions(X_combined_std, y_combined,
                      classifier=clf, test_idx=range(len(y_train), len(y_combined)))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/logistic_regression.png', dpi=300)
plt.show()

In [ ]:
clf = SGDClassifier(loss="log", alpha = 5)
clf.fit(X_train_std, y_train)

print('Accuracy: %.2f' % accuracy_score(y_test, clf.predict(X_test_std)))

plot_decision_regions(X_combined_std, y_combined,
                      classifier=clf, test_idx=range(len(y_train), len(y_combined)))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/logistic_regression.png', dpi=300)
plt.show()